In [43]:
keep_list = ['addr_state', 'annual_inc', 'application_type', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 'verification_status', 'zip_code']
print("Keep features: ", len(keep_list))

Keep features:  30


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random 
import winsound
import seaborn as sns


loan = pd.read_csv('accepted_2007_to_2018Q4.csv',low_memory=False)

In [45]:
data = loan[keep_list]
data = data.drop(['grade','sub_grade','emp_title','title','earliest_cr_line'],axis=1)

In [46]:
# not in pipeline
data['loan_amnt'] = data['loan_amnt'].astype(float)

In [47]:
# not in pipeline
data['emp_length'] = data['emp_length'].fillna('0 years')

def emp_length(x):
    if x == '10+ years':
        return 10
    if x == '< 1 year' :
        return 1
    else:
        return x.split()[0]

data['emp_length'] = data['emp_length'].astype(str).apply(lambda x:int(emp_length(x)))

In [48]:
#not in pipeline
data['term'] = data['term'].fillna('36 months')
data['term'] = data['term'].astype(str).apply(lambda x:int(x.split()[0]))

In [49]:
# not in pipeline
data[[i for i in data.columns if data[i].dtype == 'object']] =  data[[i for i in data.columns if data[i].dtype == 'object']].fillna('NA')

In [50]:
# not in pipeline
data = data[data['loan_status'].isin(['Fully Paid','Charged Off'])]
data = data[data['zip_code'] != 'NA']
data['loan_status'] = pd.get_dummies(data['loan_status'])['Fully Paid']

In [51]:
# not in pipeline 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer  = IterativeImputer()
imputer.fit(data[[i for i in data.columns if data[i].dtype != 'object']])
temp = imputer.fit_transform(data[[i for i in data.columns if data[i].dtype != 'object']])
data[[i for i in data.columns if data[i].dtype != 'object']] = temp

In [52]:
data['zip_code'] = data.zip_code.astype(str).apply(lambda x:x.replace('xx',""))
data['zip_code'] = data.zip_code.astype(int)

In [53]:
obj_cols = [i for i in data.columns if data[i].dtype == 'object']
num_cols = [i for i in data.columns if data[i].dtype != 'object']

In [54]:
num_cols.remove('loan_status')

In [55]:
x = data.drop('loan_status',axis=1)
y = data.loan_status

In [56]:
data.to_csv('Clened_data.csv',index=False)

In [57]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
enc.fit(data[obj_cols])

data[obj_cols] = enc.fit_transform(data[obj_cols])

In [58]:
from sklearn.ensemble import RandomForestClassifier


x = data.drop('loan_status',axis=1)
y = data.loan_status

forest  = RandomForestClassifier()

forest.fit(x,y)


from sklearn.metrics import f1_score

f1_score(forest.predict(x),y)

0.9999967494881604